In [1]:
import pandas as pd
import sqlite3

In [3]:
cnx = sqlite3.connect('foo.db')

In [5]:
df_customers = pd.read_sql_query("SELECT * FROM olist_customers_dataset", cnx)
df_geolocations = pd.read_sql_query("SELECT * FROM olist_geolocation_dataset", cnx)
df_orderitems = pd.read_sql_query("SELECT * FROM olist_order_items_dataset", cnx)
df_payments = pd.read_sql_query("SELECT * FROM olist_order_payments_dataset", cnx)
df_reviews = pd.read_sql_query("SELECT * FROM olist_order_reviews_dataset", cnx)
df_orders = pd.read_sql_query("SELECT * FROM olist_orders_dataset", cnx)
df_products = pd.read_sql_query("SELECT * FROM olist_products_dataset", cnx)
df_sellers = pd.read_sql_query("SELECT * FROM olist_sellers_dataset", cnx)

In [23]:
df_payments

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [21]:
df_orderitems.sort_values('order_id')

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [17]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [9]:
df_orderitems.shape

(112650, 7)

In [11]:
df_orderitems.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [ ]:
df_orders.sort_values('order_id')

In [ ]:
df_customers.to_csv('df_customers.csv', index=False)

In [ ]:
df_geolocations.to_csv('df_geolocations.csv', index=False)

In [ ]:
df_orderitems.to_csv('df_orderitems.csv', index=False)

In [ ]:
df_payments.to_csv('df_payments.csv', index=False)

In [ ]:
df_reviews.to_csv('df_reviews.csv', index=False)

In [ ]:
df_orders.to_csv('df_orders.csv', index=False)

In [ ]:
df_products.to_csv('df_products.csv', index=False)

In [ ]:
df_sellers.to_csv('df_sellers.csv', index=False)

In [ ]:
print("1")
df_customers.shape

In [ ]:
print("2")
df_geolocations.shape

In [ ]:
print("3")
df_orderitems.shape

In [ ]:
print("4")
df_payments.shape

In [ ]:
print("5")
df_reviews.shape

In [ ]:
print("6")
df_orders.shape

In [ ]:
print("7")
df_products.shape

In [ ]:
print("8")
df_sellers.shape

In [13]:
null_df_customer = df_customers.isnull().sum().sum()
null_df_geolocations = df_geolocations.isnull().sum().sum()
null_df_orderitems = df_orderitems.isnull().sum().sum()
null_df_payments = df_payments.isnull().sum().sum()
null_df_reviews = df_reviews.isnull().sum().sum()
null_df_orders = df_orders.isnull().sum().sum()
null_df_products = df_products.isnull().sum().sum()
null_df_sellers = df_sellers.isnull().sum().sum()

In [15]:
print('Number of NaN values null_df_customer:', null_df_customer)
print('Number of NaN values null_df_geolocations:', null_df_geolocations)
print('Number of NaN values null_df_orderitems:', null_df_orderitems)
print('Number of NaN values null_df_payments:', null_df_payments)
print('Number of NaN values null_df_reviews:', null_df_reviews)
print('Number of NaN values null_df_orders:', null_df_orders)
print('Number of NaN values null_df_products:', null_df_products)
print('Number of NaN values null_df_sellers:', null_df_sellers)


Number of NaN values null_df_customer: 0
Number of NaN values null_df_geolocations: 0
Number of NaN values null_df_orderitems: 0
Number of NaN values null_df_payments: 0
Number of NaN values null_df_reviews: 0
Number of NaN values null_df_orders: 0
Number of NaN values null_df_products: 0
Number of NaN values null_df_sellers: 0


In [ ]:
# Menggabungkan DataFrame
# Pertama, kita gabungkan df_orders dengan df_customers
merged_df = pd.merge(df_orders, df_customers, on='customer_id', how='inner')

# Menggabungkan dengan df_orderitems
merged_df = pd.merge(merged_df, df_orderitems, on='order_id', how='inner')

# Menggabungkan dengan df_payments
merged_df = pd.merge(merged_df, df_payments, on='order_id', how='inner')

# Menggabungkan dengan df_reviews
merged_df = pd.merge(merged_df, df_reviews, on='order_id', how='inner')

# Menggabungkan dengan df_products
merged_df = pd.merge(merged_df, df_products, on='product_id', how='inner')

# Menggabungkan dengan df_sellers
merged_df = pd.merge(merged_df, df_sellers, on='seller_id', how='inner')

# Menggabungkan dengan df_geolocations
merged_df = pd.merge(merged_df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='inner')


In [ ]:
merged_df.isna().sum()

In [ ]:
merged_df.shape

In [ ]:
# Menggabungkan df_orders dengan df_orderitems berdasarkan order_id
df = pd.merge(df_orderitems, df_orders, on='order_id', how='left')

# Menggabungkan dengan df_payments berdasarkan order_id
df = pd.merge(df, df_payments, on='order_id', how='left')

# Menggabungkan dengan df_reviews berdasarkan order_id
df = pd.merge(df, df_reviews, on='order_id', how='left')

# Menggabungkan dengan df_products berdasarkan product_id
df = pd.merge(df, df_products, on='product_id', how='left')

# Menggabungkan dengan df_sellers berdasarkan seller_id
df = pd.merge(df, df_sellers, on='seller_id', how='left')

# Menggabungkan dengan df_customers berdasarkan customer_id
df = pd.merge(df, df_customers, on='customer_id', how='left')

# Menggabungkan dengan df_geolocations berdasarkan zip_code_prefix
df = pd.merge(df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df = pd.merge(df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

In [ ]:
# Menggabungkan dengan df_customers berdasarkan customer_id
df = pd.merge(df, df_customers, on='customer_id', how='left')

In [ ]:
df = pd.merge(df, df_orderitems, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_reviews berdasarkan order_id
df = pd.merge(df_orders, df_reviews, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_payments berdasarkan order_id
df = pd.merge(df, df_payments, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_products berdasarkan product_id
df = pd.merge(df, df_products, on='product_id', how='left')

In [ ]:
# Menggabungkan dengan df_sellers berdasarkan seller_id
df = pd.merge(df, df_sellers, on='seller_id', how='left')

In [ ]:
df.isna().sum()

In [ ]:
df_combined[df_combined['customer_state'].astype(str).str.contains('DF')]

In [ ]:
df_combined

In [ ]:
#Check null Values dataframes
df_combined.isna().sum()

In [ ]:
df_null_geo = df_combined[df_combined['geolocation_city'].isna()]

In [ ]:
df_null_geo.isna().sum()

In [ ]:
df_null_pay =  df_combined[df_combined['payment_value'].isna()]

In [ ]:
df_null_pay.isna().sum()

In [ ]:
df_null_pay

In [ ]:
df_null_pay[['order_id', 'product_id', 'price', 'order_status']].value_counts('order_status')

In [ ]:
df_null_review = df_combined[df_combined['review_id'].isna()]

In [ ]:
df_nulls_drop = df_combined.dropna(subset=['review_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'])

In [ ]:
df_nulls_drop.value_counts('order_status')

In [ ]:
df_nulls_drop.shape

In [ ]:
df_null_review.value_counts('order_status')

In [ ]:
df_null_pay

In [ ]:
# Misalkan df_combined adalah DataFrame yang sudah digabungkan
# Group by 'order_status' dan hitung jumlah nilai null di 'review_score'
null_counts = df_combined.groupby('order_status')['review_score'].apply(lambda x: x.isnull().sum())

# Tampilkan hasil
null_counts


In [ ]:
df_clean = df.copy()

In [ ]:
df_clean['payment_installments'] = pd.to_numeric(df_clean['payment_installments'], errors='coerce')
df_clean['payment_value'] = pd.to_numeric(df_clean['payment_value'], errors='coerce')

#Cleansing null_values untuk data payment
df_clean['payment_installments'] = df_clean['payment_installments'].interpolate()
df_clean['payment_value'] = df_clean['payment_value'].interpolate()

In [ ]:
df_clean.isna().sum()

In [ ]:
mode_payment_type = df_clean['payment_type'].mode()[0]
df_clean['payment_type'].fillna(mode_payment_type, inplace=True)

In [ ]:
df_clean['payment_sequential'].fillna(method='ffill', inplace=True)